In [1]:
import pickle
def save_file(file_name,data):
    open_file = open(file_name, "wb")
    pickle.dump(data, open_file)
    open_file.close()

def open_file(file_name):
    open_file = open(file_name, "rb")
    Dataset = pickle.load(open_file)
    open_file.close()
    return Dataset

In [28]:
import tensorflow as tf
import random
import numpy as np

In [2]:
file_name="Dataset_clean_3000.pkl"
fedavg=open_file(file_name)

In [23]:
fedavg[0]['clients_1'][0][1]

array([5.])

In [9]:
def mnist_shuffle_data(client_percent, data_percent, num_clients):    # Get the data.
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    nb_classes= 10
    y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
    dataset, client_names= create_client(x_train, y_train, num_clients, initial='client')


    # Creating bad clients
    clients, bad_client = creating_shuffling_clients(dataset,client_names,client_percent, data_percent)

    sample_list=[clients, bad_client, x_test, y_test]
    # file_name = f"../../Data/mnist/Dataset{client_percent}_{data_percent}_{num_clients}_shuffle_mnist.pkl"
    #
    # open_file = open(file_name, "wb")
    # pickle.dump(sample_list, open_file)
    # open_file.close()
    return sample_list


In [10]:
def creating_shuffling_clients(dataset, client_names, client_percent, data_percent):
    print('this is shuffling attack')
    shards = dataset
    num_clients = len(client_names)
    random_client = random.sample(range(0, num_clients), round(num_clients * client_percent))
    random_client = np.sort(random_client)
    if len(random_client) == 0:
        print("All_ good_CLients")
    else:
        print(random_client)
        x = 0
        y = 0
        for i in range(num_clients):
            if random_client[y] == x:
                print("engaged: ", i)
                data, label = zip(*shards[i])
                label = np.array(label)
                data_length = len(label)
                random_data = random.sample(range(0, data_length), round(data_length * data_percent))
                random_data = np.sort(random_data)
                m = 0
                n = 0

                for a in range(data_length):
                    if random_data[n] == m:
                        label[a] = random.randrange(0, 9, 1)
                        if n < len(random_data) - 1:
                            n = n + 1
                    m = m + 1
                print(label)
                new_shards = list(zip(data, label))
                shards[i] = new_shards

                if y < len(random_client) - 1:
                    y = y + 1
            x = x + 1

    return {client_names[i]: shards[i] for i in range(len(client_names))}, random_client

In [12]:
def create_client(image_list, label_list, num_clients,initial='clients'):
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))
    return shards, client_names

In [17]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
nb_classes= 10
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
dataset, client_names= create_client(x_train, y_train, num_clients=100, initial='client')




In [25]:
keys_list = list(fedavg[0].keys())
values_list = list(fedavg[0].values())
print("Keys list:", keys_list)

Keys list: ['clients_1', 'clients_2', 'clients_3', 'clients_4', 'clients_5', 'clients_6', 'clients_7', 'clients_8', 'clients_9', 'clients_10', 'clients_11', 'clients_12', 'clients_13', 'clients_14', 'clients_15', 'clients_16', 'clients_17', 'clients_18', 'clients_19', 'clients_20', 'clients_21', 'clients_22', 'clients_23', 'clients_24', 'clients_25', 'clients_26', 'clients_27', 'clients_28', 'clients_29', 'clients_30', 'clients_31', 'clients_32', 'clients_33', 'clients_34', 'clients_35', 'clients_36', 'clients_37', 'clients_38', 'clients_39', 'clients_40', 'clients_41', 'clients_42', 'clients_43', 'clients_44', 'clients_45', 'clients_46', 'clients_47', 'clients_48', 'clients_49', 'clients_50', 'clients_51', 'clients_52', 'clients_53', 'clients_54', 'clients_55', 'clients_56', 'clients_57', 'clients_58', 'clients_59', 'clients_60', 'clients_61', 'clients_62', 'clients_63', 'clients_64', 'clients_65', 'clients_66', 'clients_67', 'clients_68', 'clients_69', 'clients_70', 'clients_71', 'cl

In [29]:
clients, bad_client = creating_shuffling_clients(values_list,keys_list,client_percent=.5, data_percent=1)

this is shuffling attack
[   0   10   11 ... 2993 2997 2998]
engaged:  0
[[6.]
 [2.]
 [1.]
 [2.]
 [7.]
 [1.]
 [0.]
 [2.]
 [6.]
 [2.]
 [2.]
 [1.]
 [7.]
 [5.]
 [6.]
 [0.]
 [2.]
 [2.]
 [6.]
 [7.]
 [1.]
 [5.]
 [2.]
 [3.]
 [8.]
 [2.]
 [2.]
 [5.]
 [2.]
 [2.]
 [1.]
 [4.]
 [0.]
 [7.]
 [0.]
 [7.]
 [5.]
 [4.]
 [2.]
 [6.]
 [3.]
 [4.]
 [1.]
 [3.]
 [2.]
 [2.]
 [1.]
 [4.]
 [0.]
 [2.]
 [5.]
 [1.]
 [5.]
 [7.]
 [4.]
 [1.]
 [1.]
 [8.]
 [6.]
 [1.]
 [8.]
 [0.]
 [7.]
 [2.]
 [1.]
 [1.]
 [8.]
 [5.]
 [0.]
 [6.]
 [4.]
 [7.]
 [5.]
 [1.]
 [6.]
 [5.]
 [6.]
 [7.]
 [1.]
 [2.]
 [3.]
 [5.]
 [1.]
 [8.]
 [7.]
 [1.]
 [4.]
 [5.]
 [7.]
 [3.]
 [8.]
 [5.]
 [5.]]
engaged:  10
[[1.]
 [2.]
 [3.]
 [4.]
 [4.]
 [4.]
 [0.]
 [3.]
 [0.]
 [4.]
 [4.]
 [3.]
 [6.]
 [0.]
 [7.]
 [5.]
 [3.]
 [2.]
 [1.]
 [8.]
 [1.]
 [4.]
 [5.]
 [2.]
 [8.]
 [2.]
 [6.]
 [4.]
 [4.]
 [1.]
 [4.]
 [7.]
 [7.]
 [5.]
 [8.]
 [7.]
 [4.]
 [0.]
 [6.]
 [3.]
 [0.]
 [5.]
 [7.]
 [4.]
 [2.]
 [8.]
 [0.]
 [4.]
 [0.]
 [8.]
 [7.]
 [5.]
 [3.]
 [7.]
 [8.]
 [6.]
 [1.]
 [1.]
 [4.]
 

[[7.]
 [1.]
 [8.]
 [3.]
 [1.]
 [0.]
 [1.]
 [6.]
 [1.]
 [7.]
 [4.]
 [4.]
 [2.]
 [5.]
 [4.]
 [8.]
 [5.]
 [8.]
 [6.]
 [0.]
 [3.]
 [8.]
 [8.]
 [7.]
 [7.]
 [8.]
 [8.]
 [3.]
 [5.]
 [5.]
 [3.]
 [0.]
 [6.]
 [4.]
 [5.]
 [4.]
 [8.]
 [5.]
 [6.]
 [3.]
 [1.]
 [5.]
 [6.]
 [8.]
 [7.]
 [1.]
 [7.]
 [2.]
 [3.]
 [5.]
 [4.]
 [5.]
 [2.]
 [5.]
 [7.]
 [0.]
 [0.]
 [1.]
 [8.]
 [5.]
 [0.]
 [1.]
 [8.]
 [3.]
 [0.]
 [2.]
 [7.]
 [2.]
 [1.]
 [3.]
 [8.]
 [1.]
 [7.]
 [0.]
 [4.]
 [0.]
 [1.]
 [5.]
 [0.]
 [3.]]
engaged:  196
[[7.]
 [0.]
 [4.]
 [5.]
 [8.]
 [1.]
 [6.]
 [6.]
 [2.]
 [1.]
 [7.]
 [3.]
 [8.]
 [7.]
 [8.]
 [5.]
 [3.]
 [2.]
 [7.]
 [3.]
 [8.]
 [5.]
 [8.]
 [2.]
 [5.]
 [2.]
 [0.]
 [1.]
 [5.]
 [1.]
 [8.]
 [7.]
 [3.]
 [8.]
 [4.]
 [4.]
 [8.]
 [3.]
 [4.]
 [1.]
 [0.]
 [5.]
 [4.]
 [8.]
 [4.]
 [4.]
 [3.]
 [4.]
 [2.]
 [6.]
 [3.]
 [4.]
 [3.]
 [0.]
 [5.]
 [5.]
 [5.]
 [2.]
 [6.]
 [5.]
 [7.]
 [7.]
 [8.]
 [8.]
 [5.]
 [2.]
 [1.]
 [2.]
 [5.]
 [1.]
 [7.]
 [6.]
 [3.]
 [5.]
 [1.]
 [8.]
 [5.]
 [2.]
 [4.]
 [3.]
 [4.]
 [7.]
 [5.]
 [0.]
 

[[6.]
 [4.]
 [0.]
 [3.]
 [1.]
 [3.]
 [8.]
 [8.]
 [8.]
 [6.]
 [7.]
 [1.]
 [0.]
 [4.]
 [1.]
 [4.]
 [0.]
 [7.]
 [7.]
 [5.]
 [4.]
 [5.]
 [5.]
 [6.]
 [6.]
 [4.]
 [4.]
 [8.]
 [6.]
 [6.]
 [8.]
 [0.]
 [2.]
 [5.]
 [3.]
 [7.]
 [4.]
 [3.]
 [6.]
 [2.]
 [0.]
 [8.]
 [6.]
 [2.]
 [0.]
 [2.]
 [4.]
 [6.]
 [1.]
 [2.]
 [2.]
 [6.]
 [4.]
 [5.]
 [7.]
 [8.]
 [0.]
 [3.]
 [4.]
 [5.]
 [7.]
 [2.]
 [6.]
 [3.]
 [0.]
 [7.]
 [0.]
 [6.]
 [3.]
 [2.]
 [6.]
 [5.]
 [5.]
 [2.]
 [1.]
 [3.]
 [1.]
 [6.]
 [4.]
 [1.]
 [1.]
 [2.]
 [5.]
 [2.]
 [5.]
 [7.]
 [6.]
 [5.]
 [6.]
 [6.]
 [5.]
 [4.]
 [4.]
 [1.]
 [5.]
 [4.]
 [3.]
 [5.]
 [1.]
 [2.]
 [5.]
 [0.]
 [5.]
 [7.]
 [6.]
 [8.]
 [4.]
 [8.]
 [7.]
 [4.]
 [2.]
 [2.]]
engaged:  408
[[0.]
 [4.]
 [3.]
 [2.]
 [1.]
 [5.]
 [7.]
 [1.]
 [8.]
 [8.]
 [3.]
 [2.]
 [5.]
 [1.]
 [7.]
 [2.]
 [6.]
 [3.]
 [6.]
 [8.]
 [8.]
 [0.]
 [2.]
 [6.]
 [8.]
 [8.]
 [7.]
 [4.]
 [1.]
 [7.]
 [3.]
 [4.]
 [4.]
 [5.]
 [4.]
 [7.]
 [8.]
 [7.]
 [6.]
 [6.]
 [2.]
 [8.]
 [7.]
 [2.]
 [5.]
 [0.]
 [5.]
 [7.]
 [3.]
 [8.]
 [2.]
 [1.]
 

[[4.]
 [3.]
 [5.]
 [6.]
 [7.]
 [2.]
 [7.]
 [3.]
 [4.]
 [4.]
 [4.]
 [0.]
 [2.]
 [6.]
 [7.]
 [2.]
 [2.]
 [1.]
 [5.]
 [7.]
 [7.]
 [5.]
 [2.]
 [1.]
 [2.]
 [3.]
 [4.]
 [7.]
 [7.]
 [0.]
 [6.]
 [1.]
 [8.]
 [1.]
 [2.]
 [1.]
 [0.]
 [5.]
 [4.]
 [7.]
 [6.]
 [2.]
 [7.]
 [1.]
 [4.]
 [8.]
 [7.]
 [4.]
 [3.]
 [6.]
 [3.]
 [6.]
 [8.]
 [8.]
 [8.]
 [0.]
 [8.]
 [7.]
 [8.]
 [6.]
 [1.]
 [1.]
 [0.]
 [0.]
 [4.]
 [2.]
 [4.]
 [3.]
 [4.]
 [8.]
 [1.]
 [2.]
 [2.]
 [7.]
 [1.]
 [8.]
 [0.]
 [4.]
 [1.]
 [5.]
 [6.]
 [3.]
 [2.]
 [4.]
 [5.]
 [2.]
 [3.]
 [1.]
 [7.]
 [7.]
 [2.]
 [1.]
 [8.]
 [8.]
 [2.]
 [6.]
 [0.]
 [7.]
 [8.]
 [5.]
 [6.]
 [8.]
 [2.]
 [0.]
 [3.]
 [5.]
 [5.]
 [7.]
 [3.]
 [7.]
 [5.]]
engaged:  535
[[5.]
 [1.]
 [0.]
 [0.]
 [3.]
 [8.]
 [1.]
 [1.]
 [7.]
 [3.]
 [0.]
 [7.]
 [8.]
 [0.]
 [8.]
 [4.]
 [3.]
 [5.]
 [2.]
 [8.]
 [7.]
 [4.]
 [7.]
 [3.]
 [7.]
 [6.]
 [1.]
 [1.]
 [4.]
 [3.]
 [3.]
 [4.]
 [2.]
 [5.]
 [3.]
 [0.]
 [0.]
 [4.]
 [0.]
 [8.]
 [5.]
 [3.]
 [1.]
 [8.]
 [4.]
 [3.]
 [6.]
 [2.]
 [1.]
 [3.]
 [3.]
 [6.]
 [3.]
 

[[4.]
 [5.]
 [4.]
 [7.]
 [3.]
 [0.]
 [7.]
 [2.]
 [0.]
 [0.]
 [7.]
 [7.]
 [0.]
 [5.]
 [4.]
 [8.]
 [3.]
 [3.]
 [7.]
 [2.]
 [3.]
 [8.]
 [2.]
 [7.]
 [0.]
 [7.]
 [8.]
 [1.]
 [6.]
 [3.]
 [0.]
 [1.]
 [1.]
 [6.]
 [6.]
 [8.]
 [0.]
 [1.]
 [8.]
 [6.]
 [3.]
 [5.]
 [3.]
 [2.]
 [0.]
 [5.]
 [6.]
 [5.]
 [7.]
 [4.]
 [4.]
 [4.]
 [6.]
 [3.]
 [4.]
 [2.]
 [7.]
 [4.]
 [1.]
 [6.]
 [6.]
 [8.]
 [3.]
 [4.]
 [0.]
 [3.]
 [7.]
 [8.]
 [7.]
 [7.]
 [1.]
 [7.]
 [0.]
 [8.]
 [0.]
 [8.]
 [1.]
 [6.]
 [0.]
 [0.]
 [8.]
 [4.]
 [6.]
 [2.]
 [8.]
 [2.]
 [4.]
 [6.]
 [1.]
 [8.]
 [8.]
 [4.]
 [0.]
 [1.]
 [1.]
 [8.]
 [1.]
 [8.]
 [1.]
 [3.]
 [3.]
 [7.]
 [3.]
 [6.]
 [7.]
 [8.]]
engaged:  659
[[6.]
 [6.]
 [1.]
 [2.]
 [2.]
 [6.]
 [6.]
 [8.]
 [5.]
 [8.]
 [4.]
 [0.]
 [3.]
 [4.]
 [6.]
 [4.]
 [4.]
 [1.]
 [5.]
 [6.]
 [2.]
 [1.]
 [5.]
 [4.]
 [5.]
 [7.]
 [7.]
 [0.]
 [3.]
 [0.]
 [2.]
 [7.]
 [3.]
 [3.]
 [0.]
 [5.]
 [5.]
 [0.]
 [3.]
 [6.]
 [6.]
 [6.]
 [7.]
 [2.]
 [1.]
 [0.]
 [3.]
 [8.]
 [0.]
 [4.]
 [7.]
 [2.]
 [5.]
 [0.]
 [5.]
 [0.]
 [3.]
 [4.]
 

...

 [5.]]
engaged:  2433
[[2.]
 [3.]
 [4.]
 [8.]
 [4.]
 [2.]
 [0.]
 [3.]
 [3.]
 [6.]
 [7.]
 [7.]
 [2.]
 [5.]
 [1.]
 [8.]
 [2.]
 [5.]
 [1.]
 [7.]
 [3.]
 [8.]
 [0.]
 [5.]
 [4.]
 [4.]
 [1.]
 [2.]
 [1.]
 [8.]
 [7.]
 [0.]
 [1.]
 [2.]
 [6.]
 [5.]
 [5.]
 [2.]
 [3.]
 [3.]
 [2.]
 [3.]
 [5.]
 [0.]
 [6.]
 [0.]
 [4.]
 [1.]
 [4.]
 [3.]
 [3.]
 [2.]
 [7.]
 [8.]
 [8.]
 [8.]
 [0.]
 [3.]
 [6.]
 [1.]
 [8.]
 [5.]
 [7.]
 [0.]
 [6.]
 [6.]
 [3.]
 [4.]
 [8.]
 [8.]
 [3.]
 [3.]
 [6.]
 [3.]
 [8.]
 [6.]
 [6.]
 [1.]
 [8.]
 [0.]
 [6.]
 [0.]
 [3.]
 [3.]
 [8.]
 [8.]
 [4.]
 [0.]
 [0.]
 [5.]
 [1.]
 [3.]
 [0.]]
engaged:  2435
[[8.]
 [5.]
 [7.]
 [5.]
 [2.]
 [5.]
 [7.]
 [3.]
 [3.]
 [6.]
 [7.]
 [7.]
 [6.]
 [6.]
 [7.]
 [2.]
 [1.]
 [4.]
 [3.]
 [8.]
 [7.]
 [4.]
 [2.]
 [5.]
 [2.]
 [6.]
 [6.]
 [4.]
 [5.]
 [4.]
 [8.]
 [7.]
 [0.]
 [6.]
 [0.]
 [6.]
 [6.]
 [0.]
 [6.]
 [6.]
 [7.]
 [7.]
 [5.]
 [1.]
 [4.]
 [8.]
 [4.]
 [7.]
 [7.]
 [3.]
 [5.]
 [3.]
 [4.]
 [4.]
 [0.]
 [0.]
 [3.]
 [7.]
 [1.]
 [7.]
 [6.]
 [8.]
 [6.]
 [6.]
 [7.]
 [0.]
 [4.]
 [

[[4.]
 [4.]
 [8.]
 [8.]
 [2.]
 [3.]
 [8.]
 [3.]
 [8.]
 [2.]
 [8.]
 [8.]
 [8.]
 [7.]
 [3.]
 [3.]
 [0.]
 [8.]
 [5.]
 [7.]
 [2.]
 [5.]
 [6.]
 [1.]
 [8.]
 [7.]
 [2.]
 [7.]
 [5.]
 [8.]
 [5.]
 [6.]
 [7.]
 [7.]
 [3.]
 [6.]
 [5.]
 [7.]
 [5.]
 [8.]
 [7.]
 [0.]
 [5.]
 [0.]
 [1.]
 [1.]
 [2.]
 [1.]
 [5.]
 [1.]
 [2.]
 [8.]
 [6.]
 [8.]
 [0.]
 [6.]
 [3.]
 [3.]
 [4.]
 [3.]
 [3.]
 [7.]
 [6.]
 [0.]
 [4.]
 [8.]
 [6.]
 [0.]
 [5.]
 [1.]
 [7.]
 [4.]
 [0.]
 [4.]
 [5.]
 [7.]
 [2.]
 [1.]
 [1.]
 [6.]
 [1.]
 [7.]
 [5.]
 [4.]
 [5.]
 [6.]
 [1.]
 [7.]
 [6.]
 [3.]
 [4.]
 [4.]
 [8.]
 [1.]
 [0.]]
engaged:  2623
[[7.]
 [7.]
 [0.]
 [1.]
 [6.]
 [0.]
 [8.]
 [6.]
 [1.]
 [6.]
 [5.]
 [1.]
 [3.]
 [2.]
 [1.]
 [4.]
 [0.]
 [6.]
 [8.]
 [4.]
 [2.]
 [8.]
 [8.]
 [5.]
 [6.]
 [1.]
 [4.]
 [8.]
 [7.]
 [5.]
 [7.]
 [1.]
 [0.]
 [2.]
 [0.]
 [6.]
 [2.]
 [4.]
 [2.]
 [3.]
 [2.]
 [2.]
 [7.]
 [3.]
 [1.]
 [6.]
 [3.]
 [8.]
 [3.]
 [5.]
 [0.]
 [7.]
 [5.]
 [6.]
 [1.]
 [5.]
 [5.]
 [5.]
 [5.]
 [3.]
 [8.]
 [4.]
 [4.]
 [1.]
 [8.]
 [2.]
 [7.]
 [1.]
 [7.]


 [5.]]
engaged:  2731
[[8.]
 [6.]
 [8.]
 [1.]
 [4.]
 [4.]
 [7.]
 [8.]
 [0.]
 [3.]
 [3.]
 [2.]
 [5.]
 [8.]
 [6.]
 [8.]
 [7.]
 [4.]
 [8.]
 [5.]
 [3.]
 [5.]
 [1.]
 [5.]
 [3.]
 [6.]
 [5.]
 [0.]
 [4.]
 [6.]
 [0.]
 [7.]
 [4.]
 [7.]
 [1.]
 [4.]
 [2.]
 [3.]
 [2.]
 [0.]
 [2.]
 [4.]
 [5.]
 [4.]
 [7.]
 [8.]
 [7.]
 [6.]
 [0.]
 [3.]
 [2.]
 [7.]
 [3.]
 [1.]
 [4.]
 [5.]
 [6.]
 [6.]
 [0.]
 [4.]
 [4.]
 [8.]
 [0.]
 [1.]
 [8.]
 [8.]
 [5.]
 [2.]
 [6.]
 [1.]
 [2.]
 [1.]
 [0.]
 [6.]
 [3.]
 [5.]
 [0.]
 [7.]
 [3.]
 [5.]
 [4.]
 [6.]
 [1.]
 [4.]
 [2.]
 [7.]
 [0.]
 [5.]
 [4.]
 [8.]
 [6.]
 [8.]
 [0.]
 [7.]
 [2.]
 [8.]
 [6.]
 [2.]
 [2.]
 [3.]
 [3.]
 [4.]
 [8.]
 [0.]
 [4.]
 [1.]
 [2.]]
engaged:  2732
[[6.]
 [8.]
 [7.]
 [0.]
 [3.]
 [1.]
 [4.]
 [3.]
 [0.]
 [2.]
 [1.]
 [3.]
 [0.]
 [1.]
 [0.]
 [4.]
 [2.]
 [7.]
 [6.]
 [6.]
 [4.]
 [0.]
 [6.]
 [3.]
 [2.]
 [3.]
 [2.]
 [6.]
 [3.]
 [0.]
 [2.]
 [3.]
 [2.]
 [6.]
 [6.]
 [0.]
 [6.]
 [0.]
 [7.]
 [7.]
 [8.]
 [0.]
 [1.]
 [6.]
 [6.]
 [4.]
 [1.]
 [6.]
 [3.]
 [4.]
 [0.]
 [2.]
 [1.]
 [

[[8.]
 [1.]
 [8.]
 [1.]
 [4.]
 [4.]
 [6.]
 [4.]
 [0.]
 [2.]
 [8.]
 [4.]
 [6.]
 [1.]
 [5.]
 [8.]
 [7.]
 [8.]
 [0.]
 [5.]
 [3.]
 [1.]
 [7.]
 [4.]
 [5.]
 [0.]
 [5.]
 [5.]
 [3.]
 [2.]
 [3.]
 [3.]
 [0.]
 [8.]
 [3.]
 [2.]
 [3.]
 [6.]
 [0.]
 [0.]
 [6.]
 [7.]
 [4.]
 [4.]
 [3.]
 [7.]
 [5.]
 [1.]
 [1.]
 [8.]
 [2.]
 [0.]
 [1.]
 [0.]
 [7.]
 [0.]
 [7.]
 [8.]
 [6.]
 [4.]
 [6.]
 [5.]
 [1.]
 [4.]
 [4.]
 [8.]
 [1.]
 [6.]
 [8.]
 [3.]
 [2.]
 [3.]
 [5.]
 [1.]
 [6.]
 [1.]
 [2.]
 [0.]
 [2.]
 [0.]
 [8.]
 [4.]
 [2.]
 [6.]
 [6.]
 [5.]
 [6.]
 [1.]
 [5.]
 [0.]
 [2.]
 [8.]
 [5.]
 [7.]
 [7.]
 [7.]
 [4.]
 [1.]
 [3.]
 [1.]
 [0.]
 [1.]
 [7.]
 [4.]
 [6.]
 [7.]
 [4.]
 [5.]
 [8.]
 [7.]
 [2.]
 [2.]
 [3.]
 [4.]
 [1.]
 [6.]
 [8.]]
engaged:  2840
[[4.]
 [6.]
 [0.]
 [6.]
 [3.]
 [8.]
 [0.]
 [2.]
 [0.]
 [2.]
 [2.]
 [2.]
 [0.]
 [3.]
 [5.]
 [5.]
 [1.]
 [2.]
 [6.]
 [6.]
 [7.]
 [8.]
 [0.]
 [6.]
 [1.]
 [1.]
 [6.]
 [4.]
 [6.]
 [6.]
 [6.]
 [1.]
 [4.]
 [0.]
 [1.]
 [7.]
 [7.]
 [5.]
 [5.]
 [6.]
 [5.]
 [3.]
 [8.]
 [4.]
 [6.]
 [1.]
 [4.]


[[4.]
 [0.]
 [3.]
 [8.]
 [5.]
 [6.]
 [2.]
 [6.]
 [6.]
 [2.]
 [8.]
 [5.]
 [3.]
 [2.]
 [3.]
 [5.]
 [0.]
 [3.]
 [5.]
 [1.]
 [1.]
 [7.]
 [4.]
 [4.]
 [7.]
 [8.]
 [3.]
 [0.]
 [3.]
 [7.]
 [3.]
 [6.]
 [1.]
 [2.]
 [8.]
 [0.]
 [8.]
 [0.]
 [7.]
 [5.]
 [3.]
 [3.]
 [0.]
 [4.]
 [1.]
 [2.]
 [1.]
 [1.]
 [0.]
 [2.]
 [4.]
 [1.]
 [7.]
 [3.]
 [6.]
 [1.]
 [0.]
 [8.]
 [3.]
 [7.]
 [3.]
 [5.]
 [7.]
 [6.]
 [5.]
 [1.]
 [7.]
 [8.]
 [8.]
 [7.]
 [6.]
 [5.]
 [2.]
 [7.]
 [7.]
 [3.]
 [4.]
 [1.]
 [5.]
 [6.]
 [8.]
 [5.]
 [2.]
 [6.]
 [2.]
 [1.]
 [7.]
 [8.]
 [1.]
 [8.]
 [5.]
 [0.]
 [5.]
 [6.]
 [3.]
 [2.]
 [7.]
 [7.]
 [6.]
 [4.]
 [6.]
 [3.]
 [2.]
 [3.]
 [6.]]
engaged:  2998
[[0.]
 [4.]
 [6.]
 [3.]
 [7.]
 [8.]
 [6.]
 [2.]
 [3.]
 [2.]
 [8.]
 [1.]
 [3.]
 [1.]
 [5.]
 [0.]
 [4.]
 [7.]
 [6.]
 [7.]
 [4.]
 [3.]
 [5.]
 [2.]
 [0.]
 [0.]
 [2.]
 [4.]
 [3.]
 [0.]
 [3.]
 [0.]
 [2.]
 [6.]
 [1.]
 [0.]
 [3.]
 [8.]
 [7.]
 [5.]
 [5.]
 [6.]
 [5.]
 [3.]
 [5.]
 [2.]
 [5.]
 [5.]
 [6.]
 [2.]
 [2.]
 [8.]
 [3.]
 [8.]
 [4.]
 [0.]
 [7.]
 [8.]
 [5.]


In [31]:
clients['clients_1']

[(array([1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1